In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import corner

from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
# If running on Apple Silicon or CUDA is available, use the GPU.

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# Conditional Normalizing Flows Tutorial
In this tutorial, you will work with simulated gravitational wave data and learn the posterior distribution $p(\theta|d)$ over the two black hole masses $\theta = [m_1, m_2]$. The data $d$ corresponds to the simulated strain $h_+$ (real and imaginary part), whitened with realistic detector noise of the LIGO detector: 
$$d = h_+(\theta) + n \hspace{0.5cm} \mathrm{with}~ n \sim p_{S_n}(n)$$
where the noise is taken to be a stationary Gaussian with some power spectral density $S_n$. More information about the data simulation details can be found in the notebook `01_data_generation.ipynb`.

### Data generation
Before we can start with training, we need to generate a training data set. You can find the code for this in the notebook `01_data_generation.ipynb`. You can simply run all cells in the notebook (you might need to adjust the path when saving the data).
If you are interested in gravitational wave data and how the simulation works, you can go through all explanations and visualizations provided in the data generation notebook. However, this is not necessary for the normalizing flow tutorial.

### Data loading and preprocessing for training
In this notebook, we will load and preprocess the data generated with the notebook `data_generation.ipynb` and saved as `data/training_dataset.pkl`.

In [ ]:
# Get the absolute path of the directory where the notebook is located
print("Current working directory", os.getcwd())
data_folder = os.path.join(os.getcwd(), 'genAI-Days/01_normalizing_flows/data')
file_name = os.path.join(data_folder, 'dataset.pkl')

if not os.path.isfile(file_name):
    raise ValueError(f"File {file_name} does not exist, correct path or generate data set.")

with open(file_name, 'rb') as f:
    data = pickle.load(f)
print('Sucessfully loaded dataset with', len(data['hp']), 'waveforms.')

Task 1: Inspect the contents of `data`.

Here is some general information about gravitational wave data:
A gravitational wave consists of two polarizations $h_+(\theta)$ and $h_\times(\theta)$ defined in frequency domain which are simulated based on some prior parameter values $\theta$.
In addition to the polarizations, we need information about the frequency range on which the signal is defined.

Can you extract the relevant properties from `data`?

In [ ]:
# polarization_hp, polarization_hc = np.array(...), np.array(...)
# parameters = np.array(..., dtype=np.float32)
# f_min, f_max, T = ..., ..., ...
# delta_f = ...

With this information, we can preprocess the waveforms:
- It might be the case the 0 values are saved in the polarization below $f_\mathrm{min}$. Therefore, we truncate the polarizations below $f_\mathrm{min}$.
- To simplify data handling and training, we re-package real and imaginary part of the polarization and only use $h_+$.

In [ ]:
lower_cut = int(f_min / delta_f)
waveforms = np.hstack((polarization_hp.real[:, lower_cut:], polarization_hp.imag[:, lower_cut:])).astype(np.float32)

Task 2: To make training easy for the normalizing flow, we need to standardize the parameters:

In [ ]:
# parameters_standardized = ...

Now, we wrap waveforms in a pytorch `Dataset` which returns the parameters and the signal 
$$d = h_+(\theta) + n, \hspace{0.5cm} \text{with } n \sim \mathcal{N}(0,1)$$

Task 3: Complete the code for the dataloader:

In [ ]:
class WaveformDataset(Dataset):
    def __init__(self, parameters, waveforms):
        self.parameters = parameters
        self.waveforms = waveforms

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        # params = ...
        # signal = ...
        
        # Add unit normal noise to the signal
        # noise =  ... .astype(np.float32)
        # data = ...

        if device == "mps":
            return torch.tensor(data, dtype=torch.float32, device=device), torch.tensor(params,  dtype=torch.float32, device=device)
        else:
            return torch.tensor(data, device=device), torch.tensor(params, device=device)
    
waveform_dataset = WaveformDataset(parameters_standardized, waveforms)

Task 4: Visualize the data $d$ from an arbitrary sample of the `waveform_dataset`.

In [ ]:
# Get first sample (Careful to put data back onto CPU)
# Plot signal

### Split data into train and test dataset
Task 5: Determine the number of training and test samples from a `train_fraction` and use the function `random_split()` to separate the data into a training and test dataset. Afterwards define the training and test dataloaders.

In [ ]:
train_fraction = 0.8
# num_samples = ...
# num_train = ...
# num_test = ...
train_dataset, test_dataset = random_split(waveform_dataset, [num_train, num_test])

# The DataLoader is used in training
# train_dataloader = DataLoader(...)
# test_dataloader = DataLoader(...)

### Conditional Normalizing Flow
We use the `glasflow` library. It is a wrapper for [`nflows`](https://github.com/bayesiains/nflows), which implements different flow types (e.g. autoregressive flows and coupling flows) and transforms (e.g. rational-quadratic splines).


In [ ]:

from glasflow.nflows import transforms, distributions, flows
import glasflow.nflows as nflows
import glasflow.nflows.nn.nets as nflows_nets

device = 'cpu'  # For some reason, "mps" does not work here.

The Code is adapted from examples in the [neural spline flow repository](https://github.com/bayesiains/nsf).

First, we define a function that creates the basic MAF or RQS-coupling transform.

Task 6: Understand the arguments of `create_base_transform`. Set default arguments for `hidden_dim`, `num_transform_blocks`, `batch_norm`, and `base_transform_type`. Define which activation fucntion to use and how many bins the rational-quadratic spline transformation should utilize.

In [ ]:
def create_base_transform(
    i: int,
    param_dim: int,
    context_dim: int = None,
    hidden_dim: int = ..., # TODO
    num_transform_blocks: int = ..., # TODO
    batch_norm: bool = ..., # TODO
    base_transform_type: str = ..., # TODO
):
    
    activation_fn = ... # TODO

    if base_transform_type == "maf":
        return transforms.MaskedAffineAutoregressiveTransform(
            param_dim,
            hidden_features=hidden_dim,
            context_features=context_dim,
            num_blocks=num_transform_blocks,
            activation=activation_fn,
            use_batch_norm=batch_norm,
        )
    elif base_transform_type == "rq-coupling":
        if param_dim == 1:
            mask = torch.tensor([1], dtype=torch.uint8)
        else:
            mask = nflows.utils.create_alternating_binary_mask(
                param_dim, even=(i % 2 == 0)
            )
        return transforms.PiecewiseRationalQuadraticCouplingTransform(
            mask=mask,
            transform_net_create_fn=(
                lambda in_features, out_features: nflows_nets.ResidualNet(
                    in_features=in_features,
                    out_features=out_features,
                    hidden_features=hidden_dim,
                    context_features=context_dim,
                    num_blocks=num_transform_blocks,
                    activation=activation_fn,
                    use_batch_norm=batch_norm,
                )
            ),
            num_bins= ..., # TODO
            tails="linear",
            tail_bound=1.0,
            apply_unconditional_transform=False,
        )

    else:
        raise ValueError

Next, we need to define a transform the randomly permutes the dimensions to ensure that all dimensions interact sufficiently with each other:

In [ ]:
def create_linear_transform(param_dim: int):
    return transforms.CompositeTransform(
        [
            transforms.RandomPermutation(features=param_dim),
            transforms.LULinear(param_dim, identity_init=True),
        ]
    )

Finally, we can define the `create_transform` function which composes a list of `num_flow_steps` transforms where each transform consists of a linear transform (to permute dimensions) and a base transform (MAF or RQ-coupling).
We start and finish the transforms with a linear transform that ensures that the flow outputs samples that have the correct dimensionality (`param_dim`).

Task 7: How do we have to pass the arguments of `create_transform` on to `create_linear_transform` and `create_base_transform`?

In [ ]:
def create_transform(
    num_flow_steps: int, 
    param_dim: int, 
    context_dim: int, 
    base_transform_kwargs: dict
):
    return transforms.CompositeTransform(
        [
            transforms.CompositeTransform(
                [
                    create_linear_transform(...), # TODO
                    create_base_transform(
                        ..., **base_transform_kwargs # TODO
                    ),
                ]
            )
            for i in range(num_flow_steps)
        ]
        + [create_linear_transform(...)] # TODO
    )

The entire flow model consists of a standard normal base distribution, followed by the flow transform.

Task 8: Fill in the arguments of `create_transform` and the base distribution such that it runs for the gravitational wave data:

In [ ]:
transform = create_transform(num_flow_steps = ..., # TODO
                             param_dim = ..., # TODO
                             context_dim = ..., # TODO
                             base_transform_kwargs = {"base_transform_type": ...}) # TODO

base_distribution = distributions.StandardNormal(shape=[...]) # TODO

model = flows.Flow(transform, base_distribution)
model.to(device)

### Training

Task 9: Define the loss function for training the normalizing flow based on `model.log_probs(...)` in the training and test loop.

In [ ]:
# Training and test loops

def train_loop(dataloader, model, optimizer):

    model.train()
 
    size = len(dataloader.dataset)
    train_loss = 0
    
    for batch, (X, y) in enumerate(dataloader):
        # Compute negative log probability loss  
        log_probs = ... # TODO

        loss = ... # TODO
        
        train_loss += loss.detach().sum()
        loss = loss.mean()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d} samples]")
            
    average_loss = train_loss.item() / size
    print('Average loss: {:.4f}'.format(average_loss))
    return average_loss

        
def test_loop(dataloader, model):

    model.eval()
    
    size = len(dataloader.dataset)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            log_probs  = ... # TODO
            loss = ... # TODO
            test_loss += loss.sum()

    test_loss = test_loss.item() / size
    print(f"Test loss: {test_loss:>8f} \n")
    return test_loss

Task 10: Define the parameters for the optimizer and write a loop that calls the functions `train_loop` and `test_loop` and stores the loss values in `train_history` and `test_history`:

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=..., weight_decay=...) # TODO

epochs = ... # TODO
train_history = []
test_history = []
for t in range(epochs):
    ... # TODO
print("Done!")

Task 11: Plot the training and test loss

In [ ]:
epochs = np.arange(1, len(train_history) + 1)
... # TODO

### Visualize posteriors
Finally, we can evaluate the posteriors of the trained normalizing flow based on the test dataset.
For this, we need to sample from the posterior, undo the standardization and visualize the samples in a corner plot.

Tasl 12: Complete the lines that reverse the standardization.

In [ ]:
num_posteriors = 3
num_eval_samples = 10_000

model.eval()

for n in range(num_posteriors):

    with torch.no_grad():
    
        test_x, test_y = test_dataset[n]
       
        # Sample the posterior
        test_x = test_x.expand(num_eval_samples, *test_x.shape)
        pred_samples = model.sample(1, test_x).squeeze(1).cpu().numpy()
    
        # Undo the standardization
        pred_samples = ... # TODO
        truth = ... # TODO
    
        # Plot
        corner.corner(pred_samples, truths=truth, labels=['$m_1$', '$m_2$'])
        plt.show()

### Optional Tasks:
- Visualize the distribution of masses in the training data set. What do you observe? Why is this parametrization not ideal?
- In the gravitational waves community, the two masses $m_1, m_2$ of the black holes are usually not estimated directly, but they are re-parametrized in terms of the chirp mass $\mathcal{M} = \frac{(m_1 \cdot m_2)^{3/5}}{(m_1 + m_2)^{1/5}}$ and mass ratio $q = \frac{m_2}{m_1}$. Re-define the input parameters, visualize the distribution of the training data set and re-train the flow. Does this improve training?
- The function `create_base_transform` defines a masked autoregressive flow (MAF) and a rational-quadratic spline (RQS) flow. Which flow performs better?